# Aide personnalisée au logement - logement ordinaire

In [ ]:
# CONFIGURATION DU NOTEBOOK

# Active l'affichage de résultats multiples par cellule
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
# CONFIGURATION DES FORMATS DES NOMBRES

from pandas import options

# arrondi des montants (dont plafonds de loyers, au centime d'euro le plus proche)
options.display.float_format = '{:.2f}'.format
NB_DECIMALES = 2

In [2]:
# SITUATION ANALYSÉE (AU FORMAT AIDES-SIMPLIFIÉES)

situation_18yo_moving_away = {
    "id": "18yo-moving-away",
    "description": "Un jeune de 18 ans déménage pour des études à l'université",
    "answers": {
        "statut-professionnel": "etudiant",  # activite
        "situation-professionnelle": "sans-emploi",  # dispatchSituationProfessionnelle
        "etudiant-mobilite": "parcoursup-nouvelle-region",  # dispatchEtudiantMobilite
        "boursier": True,  # boursier
        "date-naissance": "2007-03-01",  # date_naissance
        "handicap": False,  # handicap
        "statut-marital": "celibataire",  # statut_marital
        "code-postal-nouvelle-ville": "75101",  # depcom
        "situation-logement": "locataire",  # dispatchSituationLogement
        "type-logement": "logement-meuble",  # dispatchTypeLogement
        "logement-conventionne": True,  # logement_conventionne
        "colocation": False,  # coloc
        "logement-parente-proprietaire": False,  # proprietaire_proche_famille
        "nombre-personnes-logement": 1,  # 🔥 exclude: True
        "loyer-montant-mensuel": 700,  # loyer
        "loyer-montant-charges": 100,  # charges_locatives
        "loyer-difficile-payer": True,  # exclude: True
        "type-revenus": [
            "aucun-autres-revenus"
        ],
        "confirmation-end": [
            "confirmation-end-oui"
        ]
    },
    "questionsToApi": [
        "locapass-eligibilite",
        "mobilite-master-1",
        "mobilite-parcoursup",
        "aide-personnalisee-logement",
        "garantie-visale-eligibilite",
        "garantie-visale"
    ],
    "results": {
        "locapass": 1200,
        "locapass-eligibilite": True,
        "mobilite-master-1": 0,
        "mobilite-master-1-eligibilite": False,
        "mobilite-parcoursup": 500,
        "mobilite-parcoursup-eligibilite": True,
        "aide-personnalisee-logement": 327,
        "aide-personnalisee-logement-eligibilite": True,
        "garantie-visale": 800,
        "garantie-visale-eligibilite": True
    }
}

to_test = situation_18yo_moving_away

In [3]:
# DÉFINITION DES PÉRIODES DES DONNÉES ET DES CALCULS 

from notebooks.utils_mapping_simulateur import format_to_openfisca_json
from datetime import datetime
from dateutil.relativedelta import relativedelta

from openfisca_core.periods import Instant, Period

today = datetime.now()

today_year_month = today.strftime("%Y-%m")
period = today_year_month  # par exemple : '2025-04'
last_month = today - relativedelta(months=1)
period_last_month  = last_month.strftime("%Y-%m")

year = today.strftime("%Y")
last_year = (today - relativedelta(years=1)).strftime("%Y")
two_years_ago = (today - relativedelta(years=2)).strftime("%Y")
last_twelve_months = Period(('year', Instant((int(year), int(today.strftime("%m")), 1)), 1)).offset(-1).offset(-1, 'month')  # année glissante

situation_apl = format_to_openfisca_json(to_test, period)


**Extrait de la [Brochure APL 2024 (PDF)](https://www.ecologie.gouv.fr/sites/default/files/documents/Brochure-bareme-2024-APL.pdf), page 14 :**

```
APL = L + C – Pp
APL = L + C – [ P0 + Tp * ( R – R0 ) ]
APL = L + C – [ P0 + ( TF + TL ) * ( R – R0 ) ]
```

In [4]:
# CALCUL DE L'APL

from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem


tax_benefit_system = FranceTaxBenefitSystem()

sb = SimulationBuilder()
simulation = sb.build_from_entities(tax_benefit_system, situation_apl)

apl_variables = ['apl', 'logement_conventionne', 'aide_logement_montant', 'aide_logement_montant_brut', 'zone_apl']
# apl_variables_entities = ['familles', 'menages', 'familles', 'familles', 'menages']
# apl_variables_groupes = ['famille_1', 'menage_1', 'famille_1', 'famille_1', 'menage_1']

for variable_name in apl_variables: 
    print(f"{variable_name}: {simulation.calculate(variable_name, period)}")

parametres_secteur_locatif_period = tax_benefit_system.parameters(period).prestations_sociales.aides_logement.allocations_logement.locatif

# Pour mémoire, à partir d'ici, tous les simulation.calculate d'une variable impliquée dans le calcul 'apl'
# fera appel au cache des valeurs de la simulation (n'impliquera pas de re-calcul)

apl: [181.65]
logement_conventionne: [ True]
aide_logement_montant: [181.65]
aide_logement_montant_brut: [236.66]
zone_apl: ['zone_1']


In [5]:
# ANALYSE DE LA SITUATION FAMILIALE

en_couple = simulation.calculate('en_couple', period)[0]
celibataire = ~en_couple
label_situation_maritale = 'en couple' if en_couple else 'célibataire'
print(f"Situation maritale : {label_situation_maritale}")

al_nb_personnes_a_charge = simulation.calculate('al_nb_personnes_a_charge', period)[0]
print(f"Nombre de personnes à chage au sens des aides au logement : {al_nb_personnes_a_charge}")

au_moins_une_personne_a_charge = al_nb_personnes_a_charge > 0

Situation maritale : célibataire
Nombre de personnes à chage au sens des aides au logement : 0


## L - Loyer pris en compte

```

L est le loyer mensuel réel pris en compte dans la limite d’un plafond variable en fonction de trois zones géographiques et du nombre de personnes à charge.

```

In [6]:
# CONFIGURATION DE LIBRAIRIES COMPLÉMENTAIRES

from pandas import DataFrame, Index
from utils_display import affiche_resultat, colorie_reference

In [7]:
# SITUATION DU LOGEMENT

from openfisca_france.model.prestations.aides_logement import TypesZoneApl, TypesStatutOccupationLogement


zone_apl = simulation.calculate('zone_apl', period)[0]
id_zone_apl_applicable = TypesZoneApl.names[zone_apl]
print(f"Quelle est la zone APL du logement ? '{id_zone_apl_applicable}'")


print(f"\nDans le cas d'un logement en chambre, d'une location meublée, d'une colocation ou d'une sous-location, le loyer L pris en compte est réduit.")

logement_chambre = simulation.calculate('logement_chambre', period)[0]
print(f"→ Le logement est-il une chambre ? {logement_chambre}")

statut_occupation_logement = simulation.calculate('statut_occupation_logement', period)[0]
location_meuble = statut_occupation_logement == TypesStatutOccupationLogement.locataire_meuble
print(f"→ Le logement est-il une location de meublé ? {location_meuble}")

coloc = simulation.calculate('coloc', period)[0]
print(f"→ Le logement est-il une colocation ? {coloc}")

print(f"→ Le logement est-il une sous-location ? [X] TODO")


Quelle est la zone APL du logement ? 'zone_1'

Dans le cas d'un logement en chambre, d'une location meublée, d'une colocation ou d'une sous-location, le loyer L pris en compte est réduit.
→ Le logement est-il une chambre ? False
→ Le logement est-il une location de meublé ? False
→ Le logement est-il une colocation ? False
→ Le logement est-il une sous-location ? [X] TODO


In [8]:
# CALCUL DE L - LOYER PRIS EN COMPTE

loyer = simulation.calculate('loyer', period)[0]

# aide_logement_loyer_plafond (intégrant la zone_apl, la situation familiale, la colocation et le logement chambre)
# aide_logement_loyer_reel (intégrant les coefficients  de chambre et logement meublé)
aide_logement_loyer_retenu = simulation.calculate('aide_logement_loyer_retenu', period)[0]
L_calcule = round(aide_logement_loyer_retenu, NB_DECIMALES)

print(f"\n🟣 Pour un loyer déclaré de {loyer} €, le montant calculé est :\nL = {L_calcule:.2f} €")

loyer_plafond_toute_situation = None


🟣 Pour un loyer déclaré de 700.0 €, le montant calculé est :
L = 329.71 €


### Montant en euros du plafond de loyer si logement chambre (quelle que soit la situation familiale)

In [9]:
# RECUPERATION DES PARAMETRES DE PLAFOND DE LOYER PAR ZONE APL

id_zone_1 = TypesZoneApl.names[1]
id_zone_2 = TypesZoneApl.names[2]
id_zone_3 = TypesZoneApl.names[3]

parametres_locatif_zone_1 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_1]
parametres_locatif_zone_2 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_2]
parametres_locatif_zone_3 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_3]

plafond_loyer_period_zone = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_apl_applicable]

In [10]:
# TABLEAU LOGEMENT CHAMBRE

# TODO tableau complémentaire personne âgée ou handicapée adulte hébergée à titre onéreux chez des particuliers

print("🟣 Le logement est-il une chambre ?")
print(f"logement chambre : {logement_chambre}")

if logement_chambre:
    coef_chambre = parametres_secteur_locatif_period.formule.l_plafonds_loyers.coef_chambre_et_colocation.coef_chambre
    plafond_loyer_applicable_chambre = coef_chambre * plafond_loyer_period_zone.personnes_seules
    loyer_plafond_toute_situation = plafond_loyer_applicable_chambre

    data_parametres_chambre = {
        ('Logement chambre (cas général)', 'Montant'): [
            coef_chambre * parametres_locatif_zone_1.personnes_seules,
            coef_chambre * parametres_locatif_zone_2.personnes_seules,
            coef_chambre * parametres_locatif_zone_3.personnes_seules
            ]
    }

    index = Index([id_zone_1, id_zone_2, id_zone_3], name='Zone')
    df_parametres_chambre = DataFrame(data_parametres_chambre, index=index)
    df_parametres_chambre.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable_chambre)


🟣 Le logement est-il une chambre ?
logement chambre : False


### Montant en euros du plafond de loyer si colocation (quelle que soit la situation familiale)

TODO - hors périmètre simulateur déménagement aides-simplifiées à ce stade

### Montant du plafond de loyer selon situation familiale (hors logement chambre, hors colocation, hors sous-location)

In [11]:
# TABLEAU SANS PERSONNE À CHARGE

data_parametres_sans_personne_a_charge = {
    'Zone': [id_zone_1, id_zone_2, id_zone_3],
    'Personne seule': [parametres_locatif_zone_1.personnes_seules, parametres_locatif_zone_2.personnes_seules, parametres_locatif_zone_3.personnes_seules],
    'Couple': [parametres_locatif_zone_1.couples, parametres_locatif_zone_2.couples, parametres_locatif_zone_3.couples]
}

df_parametres_sans_personne_a_charge = DataFrame(data_parametres_sans_personne_a_charge)

# Définir la colonne 'Zone' comme index (optionnel, pour correspondre à l'exemple de la brochure APL)
df_parametres_sans_personne_a_charge = df_parametres_sans_personne_a_charge.set_index('Zone')

In [12]:
# TABLEAU AVEC PERSONNE À CHARGE

def get_plafond_loyer(zone_parameters, nb_personnes_a_charge):
    premier_enfant = zone_parameters.un_enfant if nb_personnes_a_charge > 0 else 0
    return premier_enfant + (zone_parameters.majoration_par_enf_supp * max(0, nb_personnes_a_charge - 1))

data_parametres_avec_personne_a_charge = {
    ('Personne seule ou couple', '1'): [parametres_locatif_zone_1.un_enfant, parametres_locatif_zone_2.un_enfant, parametres_locatif_zone_3.un_enfant],
    ('Personne seule ou couple', '2'): [get_plafond_loyer(parametres_locatif_zone_1, 2), get_plafond_loyer(parametres_locatif_zone_2, 2), get_plafond_loyer(parametres_locatif_zone_3, 2)],
    ('Personne seule ou couple', '3'): [get_plafond_loyer(parametres_locatif_zone_1, 3), get_plafond_loyer(parametres_locatif_zone_2, 3), get_plafond_loyer(parametres_locatif_zone_3, 3)],
    ('Personne seule ou couple', '4'): [get_plafond_loyer(parametres_locatif_zone_1, 4), get_plafond_loyer(parametres_locatif_zone_2, 4), get_plafond_loyer(parametres_locatif_zone_3, 4)],
    ('Personne seule ou couple', '5'): [get_plafond_loyer(parametres_locatif_zone_1, 5), get_plafond_loyer(parametres_locatif_zone_2, 5), get_plafond_loyer(parametres_locatif_zone_3, 5)],
    ('', 'Par pàc sup.'): [parametres_locatif_zone_1.majoration_par_enf_supp, parametres_locatif_zone_2.majoration_par_enf_supp, parametres_locatif_zone_3.majoration_par_enf_supp]
}

index = Index([id_zone_1, id_zone_2, id_zone_3], name='Zone')
df_parametres_avec_personne_a_charge = DataFrame(data_parametres_avec_personne_a_charge, index=index)

# Réorganiser les colonnes pour correspondre à l'ordre souhaité
df_parametres_avec_personne_a_charge = df_parametres_avec_personne_a_charge[[('Personne seule ou couple', '1'), ('Personne seule ou couple', '2'),
         ('Personne seule ou couple', '3'), ('Personne seule ou couple', '4'),
         ('Personne seule ou couple', '5'), ('', 'Par pàc sup.')]]

df_parametres_avec_personne_a_charge.columns.names = [None, 'Nombre de personnes à charge (pàc)']

In [13]:
# IDENTIFICATION DU PLAFOND DE LOYER APPLICABLE

from utils_calculate import get_latest_parameter_references


if (~logement_chambre and ~coloc):  # TODO ajouter hors sous-location
    print(f"🟣 Le ménage a-t-il des personnes à charge ? {au_moins_une_personne_a_charge}")
    if au_moins_une_personne_a_charge:
        plafond_loyer_applicable = get_plafond_loyer(plafond_loyer_period_zone, al_nb_personnes_a_charge)
        loyer_plafond_toute_situation = plafond_loyer_applicable

        print("🟣 Quel est le plafond de loyer applicable ?")
        print(f"En zone '{id_zone_apl_applicable}', pour '{al_nb_personnes_a_charge}' personne(s) à charge, le plafond de loyer qui lui est applicable est : {plafond_loyer_applicable} €")
        
        if al_nb_personnes_a_charge <= 5:
            df_parametres_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable)
        else:
            df_parametres_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(
                colorie_reference,
                reference=df_parametres_avec_personne_a_charge.loc[id_zone_apl_applicable, ''].values[0]  # valeur par pàc supplémentaire
                )
    else:
        plafond_loyer_applicable = plafond_loyer_period_zone['personnes_seules' if celibataire else 'couples']

        loyer_plafond_toute_situation = plafond_loyer_applicable
        print("🟣 Quel est le plafond de loyer applicable ?")
        print(f"En zone '{id_zone_apl_applicable}', le plafond de loyer qui lui est applicable est : {plafond_loyer_applicable} €")
        df_parametres_sans_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable)

🟣 Le ménage a-t-il des personnes à charge ? False
🟣 Quel est le plafond de loyer applicable ?
En zone 'zone_1', le plafond de loyer qui lui est applicable est : 329.71 €


In [14]:
# EXEMPLE DE REFERENCE LEGISLATIVE 

print("Quelle est la référence législative à l'origine de ces valeurs ?")
parameter_all_periods = tax_benefit_system.parameters.prestations_sociales.aides_logement.allocations_logement.locatif.formule.l_plafonds_loyers.par_zone.children[id_zone_apl_applicable].un_enfant
get_latest_parameter_references(parameter_all_periods)
# exemple d'arrêté disponible sur tricoteuses : https://legal.tricoteuses.fr/api/recherche?latest=true&q=JORFARTI000050278404

Quelle est la référence législative à l'origine de ces valeurs ?


[{'title': 'Arrêté du 27/09/2024, art. 1',
  'href': 'https://www.legifrance.gouv.fr/jorf/article_jo/JORFARTI000050278404'},
 {'title': 'Arrêté du 27/09/2019, art. 7',
  'href': 'https://www.legifrance.gouv.fr/loda/id/JORFTEXT000039160329/2024-09-29/'}]

In [15]:
from numpy import round

L_analyse = min(loyer, loyer_plafond_toute_situation)
condition_verification_L = (round(L_calcule, NB_DECIMALES) - L_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float (ici largement supérieure)

print(f"🟣 Le loyer retenu doit être le minimum entre le loyer du logement {loyer} et le loyer plafond {loyer_plafond_toute_situation}, soit : {L_analyse}")
affiche_resultat("L", "Le loyer calculé pour l'APL correspond-t-il bien à l'attendu ?", condition_verification_L, round(L_calcule, NB_DECIMALES))

🟣 Le loyer retenu doit être le minimum entre le loyer du logement 700.0 et le loyer plafond 329.71, soit : 329.71
✅ Le loyer calculé pour l'APL correspond-t-il bien à l'attendu ?
L : 329.71


## C - Forfait charges pris en compte

In [16]:
aide_logement_charges = simulation.calculate('aide_logement_charges', period)[0]
C_calcule = aide_logement_charges

forfait_charges_toute_situation = None

# paramètres forfait charges - cas général
montant_forfaitaire_charges_seul_ou_couple = parametres_secteur_locatif_period.formule.c_forfait_charges.cas_general.cas_general
montant_majoration_charges_par_enfant = parametres_secteur_locatif_period.formule.c_forfait_charges.cas_general.majoration_par_enfant

In [17]:
# TABLEAU SANS PERSONNE À CHARGE - CAS GÉNÉRAL
# TODO cas des colocataires

if (~au_moins_une_personne_a_charge):
    fofait_charges_applicable = montant_forfaitaire_charges_seul_ou_couple
    forfait_charges_toute_situation = fofait_charges_applicable

    data_parametres_charges_sans_personne_a_charge = {
        'Personne seule ou en couple': [montant_forfaitaire_charges_seul_ou_couple]
    }

    index = Index(['Cas général'], name='Cas')
    df_parametres_charges_sans_personne_a_charge = DataFrame(data_parametres_charges_sans_personne_a_charge, index=index)
    
    print("Sans personne à charge, la législation indique ce forfait charges :")
    df_parametres_charges_sans_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=fofait_charges_applicable)

Sans personne à charge, la législation indique ce forfait charges :


In [18]:
# TABLEAU AVEC PERSONNE À CHARGE
# TODO cas des colocataires


if au_moins_une_personne_a_charge:
    forfait_charges_applicable = montant_forfaitaire_charges_seul_ou_couple + al_nb_personnes_a_charge * montant_majoration_charges_par_enfant
    forfait_charges_toute_situation = forfait_charges_applicable

    data_parametres_charges_avec_personne_a_charge = {
        ('Personne seule ou couple', '1'): [montant_forfaitaire_charges_seul_ou_couple + montant_majoration_charges_par_enfant],
        ('Personne seule ou couple', '2'): [montant_forfaitaire_charges_seul_ou_couple + 2 * montant_majoration_charges_par_enfant],
        ('Personne seule ou couple', '3'): [montant_forfaitaire_charges_seul_ou_couple + 3 * montant_majoration_charges_par_enfant],
        ('', 'Par pàc sup.'): [montant_majoration_charges_par_enfant]
    }

    index = Index(['Cas général'], name='Cas')
    df_parametres_charges_avec_personne_a_charge = DataFrame(data_parametres_charges_avec_personne_a_charge, index=index)

    # Réorganiser les colonnes pour correspondre à l'ordre souhaité
    df_parametres_charges_avec_personne_a_charge = df_parametres_charges_avec_personne_a_charge[[
        ('Personne seule ou couple', '1'),
        ('Personne seule ou couple', '2'),
        ('Personne seule ou couple', '3'),
        ('', 'Par pàc sup.')]]

    df_parametres_charges_avec_personne_a_charge.columns.names = [None, 'Nombre de personnes à charge (pàc)']

    print("À partir d'au moins 1 personne à charge, la législation applique ces forfaits charges :")
    if al_nb_personnes_a_charge <= 3:
        df_parametres_charges_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=forfait_charges_applicable)
    else:
        df_parametres_charges_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=montant_majoration_charges_par_enfant)

In [19]:
C_analyse = forfait_charges_toute_situation
condition_verification_C = (round(C_calcule, NB_DECIMALES) - C_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float (ici largement supérieure)

print(f"🟣 Le forfait charges retenu dépend de la situation maritale '{label_situation_maritale}' et du nombre '{al_nb_personnes_a_charge}' de personnes à charges, soit : {C_analyse}")
affiche_resultat("C", "Le forfait charges calculé pour l'APL correspond-t-il bien à l'attendu ?", condition_verification_C, round(C_calcule, NB_DECIMALES))

🟣 Le forfait charges retenu dépend de la situation maritale 'célibataire' et du nombre '0' de personnes à charges, soit : 59.97
✅ Le forfait charges calculé pour l'APL correspond-t-il bien à l'attendu ?
C : 59.97


## P0 - Participation personnelle minimale

```
P0 est une composante de la participation personnelle Pp qui correspond à la part fixe, indépendante du revenu, que doit au minimum acquitter le locataire.
```

In [20]:
p0_taux = parametres_secteur_locatif_period.formule.pp_particip_perso.p0_particip_min.p0_taux
p0_forfait = parametres_secteur_locatif_period.formule.pp_particip_perso.p0_particip_min.p0_forfait

depense_logement_plafonnee_calculee = L_calcule + C_calcule
depense_logement_plafonnee_analyse = L_analyse + C_analyse

p0_analyse = max(p0_forfait, p0_taux * depense_logement_plafonnee_analyse)
p0_calcule = max(p0_forfait, p0_taux * depense_logement_plafonnee_calculee)  # P0 est inclue dans aide_logement_participation_personnelle
condition_verification_P0 = (round(p0_calcule, NB_DECIMALES) - p0_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float (ici largement supérieure)

print("🟣 La participation personnelle minimale doit être la plus élevée des deux valeurs suivantes :")
print(f"* {p0_taux * 100} % de la dépense de logement plafonnée (L + C = {L_analyse} + {C_analyse}) soit : {round(p0_taux * depense_logement_plafonnee_analyse, NB_DECIMALES)}")
print(f"* {p0_forfait} € (montant forfaitaire) \n")
affiche_resultat("P0", "La participation personnelle minimale calculée dont doit s'acquitter le locataire correspond-t-elle bien à l'attendu ?", condition_verification_P0, round(p0_calcule, NB_DECIMALES))

🟣 La participation personnelle minimale doit être la plus élevée des deux valeurs suivantes :
* 8.5 % de la dépense de logement plafonnée (L + C = 329.71 + 59.97) soit : 33.12
* 39.15 € (montant forfaitaire) 

✅ La participation personnelle minimale calculée dont doit s'acquitter le locataire correspond-t-elle bien à l'attendu ?
P0 : 39.15


## Ressources

### R - Ressources annuelles

**Extrait de la [Brochure APL 2024 (PDF)](https://www.ecologie.gouv.fr/sites/default/files/documents/Brochure-bareme-2024-APL.pdf), page 60 :**

Les ressources à prendre en compte depuis le 1er janvier 2021 sont celles perçues par l’allocataire, son conjoint ou concubin, ainsi que par toutes les personnes ayant vécu au moins six mois au foyer du bénéficiaire au cours de la période de référence [M-13 ; M-2] et s’y trouvant encore au moment de la demande ou au début de la période de paiement.
Les périodes de référence de prise en compte des ressources sont multiples :
* [M-13 ; M-2] pour les données (salaires et traitements notamment) déclarées par les employeurs dans le cadre de la déclaration sociale nominative (DSN, définie à l’article L. 133-5-3 du Code de la sécurité sociale) ou par les organismes verseurs (notamment pour les revenus de remplacement comme les indemnités journalières, les allocations chômage ou les pensions de retraite) dans le cadre du prélèvement à la source pour l’impôt sur le revenu, les revenus d’activité perçus hors de France ou versés par une organisation internationale, et le chiffre d’affaires des travailleurs indépendants ayant commencé leur activité après le 1er janvier N-2 ;
* N-1 pour les pensions alimentaires versées ou reçues, les frais de tutelle et les frais professionnels (lorsque ces derniers excèdent la déduction forfaitaire de 10 % prévue au 3° de l’article 83 du Code général des impôts). Dans le cas où le montant de pensions alimentaires reçues en N-1 n’est pas connu, la valeur N-2 est prise en compte par défaut ;
* N-2 pour les autres revenus imposables et notamment les revenus des travailleurs indépendants ayant débuté leur activité le ou avant le 1er janvier N-2 (ayant donc une activité d’indépendant sur l’ensemble de l’année N-2).

Ces périodes de référence s’apprécient par rapport au mois M d’ouverture ou de réexamen trimestriel du droit.

Par exemple, pour un droit recalculé pour la période trimestrielle de décembre 2023 / janvier 2024 / février 2024, le mois M de réexamen correspond au mois de décembre 2023. Ainsi :
* la période [M-13 ; M-2] correspond aux mois de novembre 2022 à octobre 2023 ;
* la période N-1 correspond à l’année 2022 ;
* la période N-2 correspond à l’année 2021.

> Pour en savoir plus sur le périmètre précis des ressources, les charges déductibles, la prise en compte du patrimoie, les mesures d'abattement et de neutralisation sociaux et les spécificités territoriales voir les pages 60 à 66 de la brochure. 📚
> 
> Extrait (page 61) :
> 
> Les ressources servant au calcul de l’allocation de logement et de l’aide personnalisée au logement s’entendent du total des **revenus nets catégoriels** retenus pour l’établissement de l’impôt sur le revenu, après prise en compte de certaines déductions.
> Par revenus nets catégoriels, sont désignées les différentes catégories de revenus indiquées à l’article 1er du Code général des impôts (CGI), affectées des abattements, déductions et majorations afférents à chacune d’entre elles.

In [21]:
# DETAIL DES RESSOURCES PRISES EN COMPTE POUR L'APL

# pour les ressources, on affiche les valeurs de tous les individus connus de la simulation
print("Quelles sont les ressources connues de toutes les personnes connues de la simulation ?")

aide_logement_base_ressources = simulation.calculate('aide_logement_base_ressources', period)
print(f"aide_logement_base_ressources = {aide_logement_base_ressources}")

aide_logement_base_ressources_individu = simulation.calculate_add('aide_logement_base_ressources_individu', last_twelve_months)  # personnes du logement, éventuellement parents demandeur
print(f"aide_logement_base_ressources_individu = {aide_logement_base_ressources_individu}")

salaire_imposable = simulation.calculate_add('salaire_imposable', last_twelve_months)
print(f"salaire_imposable = {salaire_imposable}")

chomage_imposable = simulation.calculate_add('chomage_imposable', last_twelve_months)
print(f"chomage_imposable = {chomage_imposable}")

bourse_college = simulation.calculate('bourse_college', period)
print(f"bourse_college = {bourse_college}")

bourse_lycee = simulation.calculate('bourse_lycee', period)
print(f"bourse_lycee = {bourse_lycee}")

bourse_criteres_sociaux = simulation.calculate('bourse_criteres_sociaux', period)
print(f"bourse_criteres_sociaux = {bourse_criteres_sociaux}")

bourse_enseignement_sup = simulation.calculate('bourse_enseignement_sup', period)
print(f"bourse_enseignement_sup = {bourse_enseignement_sup}")

aide_logement_base_ressources_patrimoine = simulation.calculate('aide_logement_base_ressources_patrimoine', period)
print(f"aide_logement_base_ressources_patrimoine = {aide_logement_base_ressources_patrimoine}")

aide_logement_base_revenus_fiscaux = simulation.calculate('aide_logement_base_revenus_fiscaux', two_years_ago)
print(f"aide_logement_base_revenus_fiscaux = {aide_logement_base_revenus_fiscaux}")

pensions_alimentaires_versees = simulation.calculate('pensions_alimentaires_versees', last_year)
print(f"pensions_alimentaires_versees = {pensions_alimentaires_versees}")

# TODO ? :
# abattements_speciaux_prestations_familiales N-2 
# biactivite

Quelles sont les ressources connues de toutes les personnes connues de la simulation ?
aide_logement_base_ressources = [8600.]
aide_logement_base_ressources_individu = [0.]
salaire_imposable = [0.]
chomage_imposable = [0.]
bourse_college = [0.]
bourse_lycee = [0.]
bourse_criteres_sociaux = [0.]
bourse_enseignement_sup = [0.]
aide_logement_base_ressources_patrimoine = [0.]
aide_logement_base_revenus_fiscaux = [0.]
pensions_alimentaires_versees = [0.]


In [22]:
# MONTANTS PLANCHERS DE RESSOURCES (ÉTUDIANTS, BOURSIERS)

print("Il existe des montants planchers de ressources pris en compte pour les étudiants et boursiers.")
etudiant = simulation.calculate('etudiant', period)[0]
boursier = simulation.calculate('boursier', period)[0]

if etudiant:
    forfait_ressources_etudiant = tax_benefit_system.parameters(period).prestations_sociales.aides_logement.allocations_logement.ressources.etudiants.locatif.dar_4_forfait_ressources
    print(f"L'individu est etudiant, le montant plancher de ressources sur les 12 derniers mois est : {forfait_ressources_etudiant}")

if boursier:
    forfait_ressources_boursier = tax_benefit_system.parameters(period).prestations_sociales.aides_logement.allocations_logement.ressources.etudiants.locatif.dar_5_minoration_boursier
    print(f"L'individu est boursier, le montant plancher de ressources sur les 12 derniers mois est : {forfait_ressources_boursier}")

Il existe des montants planchers de ressources pris en compte pour les étudiants et boursiers.
L'individu est etudiant, le montant plancher de ressources sur les 12 derniers mois est : 8600


In [23]:
# EVALUATION DES RESSOURCES R

R_calcule = aide_logement_base_ressources[0]
print(f"🟣 Les ressources prises en compte sont :\nR = {R_calcule}")  # pas d'analyse à émettre sur les ressources

🟣 Les ressources prises en compte sont :
R = 8600.0


## R0 - Abattement forfaitaire 

**Extrait de la [Brochure APL 2024 (PDF)](https://www.ecologie.gouv.fr/sites/default/files/documents/Brochure-bareme-2024-APL.pdf), page 17 :**

R0 correspond à un abattement forfaitaire appliqué aux ressources des ménages. Il s’agit par ailleurs du seuil de ressources annuelles au-delà duquel l’aide commence à décroitre. Autrement dit, un ménage dont les ressources annuelles sont inférieures ou égales au R0 bénéficie d’une aide personnelle au logement maximale.

In [24]:
# IDENTIFICATION DE L'ABATTEMENT FORFAITAIRE R0

parameteres_r0 = parametres_secteur_locatif_period.formule.pp_particip_perso.r0_abattement.cas_general

r0_analyse = None
if not au_moins_une_personne_a_charge:
    r0_analyse = parameteres_r0.taux_seul if celibataire else parameteres_r0.taux_couple
else:
    if al_nb_personnes_a_charge <= 6:
        r0_analyse = parameteres_r0.children[f"taux{al_nb_personnes_a_charge}pac"]
    else:
        r0_analyse = parameteres_r0.taux6pac + (al_nb_personnes_a_charge - 6) * parameteres_r0.taux_pac_supp

data_parametres_abattement_forfaitaire = {
        ('Personne seule', '0'): [parameteres_r0.taux_seul],
        ('Couple', '0'): [parameteres_r0.taux_couple],
        ('Personne seule ou couple', '1'): [parameteres_r0.taux1pac],
        ('Personne seule ou couple', '2'): [parameteres_r0.taux2pac],
        ('Personne seule ou couple', '3'): [parameteres_r0.taux3pac],
        ('Personne seule ou couple', '4'): [parameteres_r0.taux4pac],
        ('Personne seule ou couple', '5'): [parameteres_r0.taux5pac],
        ('Personne seule ou couple', '6'): [parameteres_r0.taux6pac],
        ('', 'Par pàc sup.'): [parameteres_r0.taux_pac_supp]
    }

index = Index(['Composition du ménage bénéficiaire'], name='Nombre de personnes à charge')
df_parametres_abattement_forfaitaire = DataFrame(data_parametres_abattement_forfaitaire, index=index)

# Réorganiser les colonnes pour correspondre à l'ordre souhaité
df_parametres_abattement_forfaitaire = df_parametres_abattement_forfaitaire[[
    ('Personne seule', '0'),
    ('Couple', '0'),
    ('Personne seule ou couple', '1'),
    ('Personne seule ou couple', '2'),
    ('Personne seule ou couple', '3'),
    ('Personne seule ou couple', '4'),
    ('Personne seule ou couple', '5'),
    ('Personne seule ou couple', '6'),
    ('', 'Par pàc sup.')]]

print(f"🟣 L'abattement forfaitaire dépend de la composition familiale : {r0_analyse} pour '{al_nb_personnes_a_charge}' personne(s) à charge")

if al_nb_personnes_a_charge <= 6:
    df_parametres_abattement_forfaitaire.T.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=r0_analyse)
else:
    df_parametres_abattement_forfaitaire.T.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=parameteres_r0.taux_pac_supp)

r0_calcule = simulation.calculate('aide_logement_R0', period)[0]
condition_verification_R0 = (round(r0_calcule, NB_DECIMALES) - r0_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float
affiche_resultat("R0", "L'abattement forfaitaire calculé correspond-t-il bien à l'attendu ?", condition_verification_R0, round(r0_calcule, NB_DECIMALES))

🟣 L'abattement forfaitaire dépend de la composition familiale : 5235 pour '0' personne(s) à charge
✅ L'abattement forfaitaire calculé correspond-t-il bien à l'attendu ?
R0 : 5235.00


In [25]:
# RESSOURCES APRÈS ABATTEMENT

print(f"🟣 Quel est le montant de ressources calculé après abattement ?")
print(f"R - R0 = {R_calcule} - {r0_calcule} = {R_calcule - r0_calcule} €")


🟣 Quel est le montant de ressources calculé après abattement ?
R - R0 = 8600.0 - 5235.0 = 3365.0 €


## TP - Taux de participation personnelle 

**Extrait de la [Brochure APL 2024 (PDF)](https://www.ecologie.gouv.fr/sites/default/files/documents/Brochure-bareme-2024-APL.pdf), page 18 :**

Le taux de participation personnelle TP est la somme des taux TF et TL avec :
- TF, taux fonction de la taille du ménage ;
- TL, taux complémentaire fonction du loyer.

In [26]:
# TODO métropole analysé - ajouter 'dom' 

parametres_tp = parametres_secteur_locatif_period.formule.pp_particip_perso.tp_taux.tf_taille_famille.metropole


### TF - Taux fonction de la taille du ménage

In [27]:
# TABLEAU TF

tf_analyse = 100 * parametres_tp.personnes_isolees  # TODO

data_parametres_tf = {
        ('Personne seule', '0'): [100 * parametres_tp.personnes_isolees],
        ('Couple', '0'): [100 * parametres_tp.couples_0_enfant],
        ('Personne seule ou couple', '1'): [100 * parametres_tp.avec_1_enfant],
        ('Personne seule ou couple', '2'): [100 * parametres_tp.avec_2_enfants],
        ('Personne seule ou couple', '3'): [100 * parametres_tp.avec_3_enfants],
        ('Personne seule ou couple', '4'): [100 * parametres_tp.avec_4_enfants],
        ('Personne seule ou couple', '5'): [100 * parametres_tp.avec_5_enfants],
        ('Personne seule ou couple', '6'): [100 * parametres_tp.avec_6_enfants],
        ('', 'Par pàc sup.'): [str(100 * parametres_tp.maj_par_enf_supp)]  # évite l'arrondi de ce paramètre à +2 décimales
    }

index = Index(['Taux fonction de la taille du ménage (TF)'], name='Nombre de personnes à charge')
df_parametres_tf = DataFrame(data_parametres_tf, index=index)

# Réorganiser les colonnes pour correspondre à l'ordre souhaité
df_parametres_tf = df_parametres_tf[[
    ('Personne seule', '0'),
    ('Couple', '0'),
    ('Personne seule ou couple', '1'),
    ('Personne seule ou couple', '2'),
    ('Personne seule ou couple', '3'),
    ('Personne seule ou couple', '4'),
    ('Personne seule ou couple', '5'),
    ('Personne seule ou couple', '6'),
    ('', 'Par pàc sup.')]]

print(f"🟣 Le taux fonction de la taille du ménage attendu est : {tf_analyse} pour '{al_nb_personnes_a_charge}' personne(s) à charge")

if al_nb_personnes_a_charge <= 6:
    df_parametres_tf.T.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=tf_analyse)
else:
    df_parametres_tf.T.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=parametres_tp.maj_par_enf_supp)

aide_logement_taux_famille = simulation.calculate('aide_logement_taux_famille', period)[0]  # TF
tf_calcule = aide_logement_taux_famille

condition_verification_TF = (round(tf_calcule, NB_DECIMALES) - tf_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float
affiche_resultat(
    "TF",
    "Le taux fonction de la taille du ménage calculé correspond-t-il bien à l'attendu ?",
    condition_verification_TF,
    round(tf_calcule, NB_DECIMALES))

🟣 Le taux fonction de la taille du ménage attendu est : 2.83 pour '0' personne(s) à charge
✅ Le taux fonction de la taille du ménage calculé correspond-t-il bien à l'attendu ?
TF : 0.03


### TL - Taux complémentaire fonction du loyer

**Extrait de la [Brochure APL 2024 (PDF)](https://www.ecologie.gouv.fr/sites/default/files/documents/Brochure-bareme-2024-APL.pdf), page 18 :**

TL dépend du rapport RL :
* du loyer réel éventuellement plafonné (L),
* sur le loyer de référence (LR) qui correspond au loyer plafond de la zone II métropole pour le ménage considéré en locatif ordinaire.

RL = L / LR
RL est exprimé en pourcentage et est arrondi à la deuxième décimale.

In [28]:
L_analyse
L_calcule

LR = parametres_locatif_zone_2.couples  # TODO vérifier : ou personnes_seules ?
LR

RL_analyse = L_analyse / LR 
RL_calcule = L_calcule / LR

In [29]:
# IDENTIFICATION DU TP CALCULE

aide_logement_taux_loyer = simulation.calculate('aide_logement_taux_loyer', period)[0]  # TL 
tp_calcule = aide_logement_taux_famille + aide_logement_taux_loyer

condition_verification_TP = (round(tp_calcule, NB_DECIMALES) - tf_analyse) <= 0.01  # marge d'erreur pour écarts numpy.float32 et float (! ici plus faible qu'ailleurs)

affiche_resultat(
    "TP",
    "Le taux de participation personnelle calculé correspond-t-il bien à l'attendu ?",
    condition_verification_TP,
    round(tp_calcule, NB_DECIMALES))



✅ Le taux de participation personnelle calculé correspond-t-il bien à l'attendu ?
TP : 0.03
